In [2]:
import json

notebook_file = "/home/ryounis/Documents/Zurich/PEACHLab/datascience-visualisation/data/datasets/Natural Language Processing with Disaster Tweets/classified_notebooks.json"
with open(notebook_file, 'r') as file:
    data = json.load(file)

In [5]:
data

{'notebooks': [{'cells': {'cells': [{'cell_id': 0,
      'code': '# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the

In [6]:
import os
from tqdm import tqdm
import sys; sys.path.insert(0, "../")
from Clusterers.clusterer import ClassCluster
with open('../../../secrets/api_key.txt', 'r') as f: api_key = f'{f.read()}'

os.environ["OPENAI_API_KEY"] = api_key
clusterer = ClassCluster()
for notebook in tqdm(data["notebooks"]):
    for cell in notebook["cells"]["cells"]:
        cell["embedding"] = clusterer.embed_cell(cell["code"], cell["desc"])

100%|██████████| 28/28 [02:31<00:00,  5.42s/it]
